<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/LangChain_%2B_Vicuna_7b_v1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考　# https://note.com/npaka/n/nb3c02ce2d4c5　など

In [1]:
# パッケージのインストール
!pip install -q langchain accelerate bitsandbytes sentence_transformers
!pip install -q langchain_experimental openai python-dotenv duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 1.4 MB/s eta 0:00:0

In [2]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [3]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
wrapper = DuckDuckGoSearchAPIWrapper(region="wt-wt", max_results=3)

In [4]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults(api_wrapper=wrapper)
results = search.run("ぼっち・ざ・ろっく!")
results.split("], [")[0]

'[snippet: COMICS TWITTER SHARE SCROLL MOVIE 2023.5.21 劇場総集編 WEB予告 2022.12.25 TVアニメ「ぼっち・ざ・ろっく! 」本PV 2022.11.6 TVアニメ「ぼっち・ざ・ろっく! 」第2弾PV 2022.9.14 【第1弾PV】「ぼっち・ざ・ろっく! 」2022年10月8日㈯より放送開始! 2021.12.18 TVアニメ「ぼっち・ざ・ろっく! 」2022年放送開始! SEE ALL, title: TVアニメ「ぼっち・ざ・ろっく!」公式サイト, link: https://bocchi.rocks/'

In [5]:
# モデルをダウンロード
!wget -c https://huggingface.co/TheBloke/vicuna-7B-v1.5-GGML/resolve/main/vicuna-7b-v1.5.ggmlv3.q4_K_M.bin

--2023-08-18 13:04:39--  https://huggingface.co/TheBloke/vicuna-7B-v1.5-GGML/resolve/main/vicuna-7b-v1.5.ggmlv3.q4_K_M.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.102, 13.33.33.110, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/04/4a/044aeab26287930d4ce031dd7ef9a573071cbc19c70dca20e5019f183d08627b/d5c5d43300f9f6a4e9e9d88d43f958e7fd2898284cc12ad0934980d591a3537e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vicuna-7b-v1.5.ggmlv3.q4_K_M.bin%3B+filename%3D%22vicuna-7b-v1.5.ggmlv3.q4_K_M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1692623080&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MjYyMzA4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8wNC80YS8wNDRhZWFiMjYyODc5MzBkNGNlMDMxZGQ3ZWY5YTU3MzA3MWNiYzE5YzcwZGNhMjBlNTAxOWYxOD

In [6]:
# llma-cpp-pythonをインストール
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 210.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 176.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822279 sha256=72ff1c75e51ef3c07ba9fa864df625817fbafbac63793a6f8b5d5e6264a4b32a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gac_ftzy/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    U

In [7]:
# LLM
from llama_cpp import Llama
from langchain.llms import LlamaCpp

# LLMの準備
n_gpu_layers = 32
n_batch = 512

llm = LlamaCpp(
    model_path="/content/vicuna-7b-v1.5.ggmlv3.q4_K_M.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4048,
    temperature=0.25,
    max_tokens=512,
    top_p=1,
    # verbose=True,
)

# プロンプトの準備
prompt = """### Instruction: What is the height of Mount Fuji?
### Response:"""

# 推論の実行
output = llm(
    prompt,
    temperature=0.1,
    stop=["Instruction:", "Input:", "Response:", "\n"],
    echo=True,
)

# 結果の表示
print(output)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


 The height of Mount Fuji is approximately 12,388 feet (3,764 meters).


In [8]:
from langchain.agents import load_tools

# ツールの準備
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for when you need to get infomation by queries."
    ),
]

In [9]:
from langchain.agents import initialize_agent
# from langchain.agents import AgentType

# エージェントの作成
agent = initialize_agent(
    agent="zero-shot-react-description",
    llm=llm,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_tokens=100
)

In [10]:
response = agent.run("アニメ「ぼっち・ざ・ろっく」主人公が通う学校の名前は？")
print(response)



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 The school's name is important for understanding the context of the anime.
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 I need to provide a search query to find the answer.
Action: Search
Action Input: "アニメ「っち・・ろっく」主人公が通う学校の名前" school name
Observation: [snippet: そして予告された7月18日に「い・ろ・は・す」と「ぼっち・ざ・ろっく!」のコラボレーションが告知され、テレビアニメの既存の映像に新規カットを加えるなどして再構成した映像に新たにアフレコを行った動画がTwitterとYouTubeで公開され、Twitterでは ..., title: ぼっち・ざ・ろっく! (アニメ) - Wikipedia, link: https://ja.wikipedia.org/wiki/ぼっち・ざ・ろっく!_(アニメ)], [snippet: 『ブルーロック』（bluelock）は、金城宗幸（原作）、ノ村優介（作画）による日本の漫画。『週刊少年マガジン』（講談社）にて2018年35号より連載中 。 2021年5月に第45回講談社漫画賞の少年部門を受賞 。 2023年7月時点で累計部数は2700万部を突破している 。. スピンオフ作品として、『別冊少年 ..., title: ブルーロック - Wikipedia, link: https://ja.wikipedia.org/wiki/ブルーロック], [snippet: アニメ『ぼっち・ざ・ろっく!』第2期が制作される可能性や、原作マンガの何巻までアニメ化されたのかなどの情報をまとめています。2022年秋に放送された第1期では、12話で原作漫画2巻の途中までのストーリーが映像化されました。, title: アニメ『ぼっち・ざ・ろっく!』 2期はいつから？続編の可能性はある？ | インタビュー・特集 | Abema Times | アベマタイムズ, link: https://times.abema.tv/articles/-/10065124], [snippet: hatena Pocket 目次を表示 引っ込み思案の女子高生がバンドを通して成長していく姿を描いた作品「ぼっち・ざ・ろっく! 」。 2022年にはテレビアニメが放送され、劇中で登場した楽曲が人気となったほか、多くのファンアートも生み出されてい

...

Llama.generate: prefix-match hit


 The school's name is still not clear from the search results.
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 I need to provide a more specific search query for the school's name.
Action: Search
Action Input: "アニメ「っち・・ろっく」主人公が通う学校の名前" school name
Observation: [snippet: そして予告された7月18日に「い・ろ・は・す」と「ぼっち・ざ・ろっく!」のコラボレーションが告知され、テレビアニメの既存の映像に新規カットを加えるなどして再構成した映像に新たにアフレコを行った動画がTwitterとYouTubeで公開され、Twitterでは ..., title: ぼっち・ざ・ろっく! (アニメ) - Wikipedia, link: https://ja.wikipedia.org/wiki/ぼっち・ざ・ろっく!_(アニメ)], [snippet: 劇中に登場するバンド「結束バンド」の楽曲を収録したアルバムは、アルバムランキングで首位を獲得したり、主人公の後藤ひとり（ごとう ひとり）が持っているモデルのギターが売れるなどアニメの枠を飛び越えて話題になっています。 この記事では、『ぼっち・ざ・ろっく! 』第2期が制作される可能性や、原作マンガの何巻までアニメ化されたのかなどの情報をまとめています。 目次 『ぼざろ』映画化! 1期の劇場総集編の上映が決定 アニメ『ぼっち・ざ・ろっく! 』とは？ アニメ『ぼっち・ざ・ろっく! 』2期が制作される可能性は？ アニメ『ぼっち・ざ・ろっく! 』2期はいつから？ 2期のネット配信（サブスク配信）はある？ 目次を開く, title: アニメ『ぼっち・ざ・ろっく!』 2期はいつから？続編の可能性はある？ | インタビュー・特集 | Abema Times | アベマタイムズ, link: https://times.abema.tv/articles/-/10065124], [snippet: 『ブルーロック』（bluelock）は、金城宗幸（原作）、ノ村優介（作画）による日本の漫画。『週刊少年マガジン』（講談社）にて2018年35号より連載中 。 2021年5月に第45回講談社漫画賞の少年部門を受賞 。 2023年7月時点で累計部数は2700万部を突破している 。. スピンオフ作品として、『別冊

ValueError: ignored

In [11]:
%time
response = agent.run("ぼっち・ざ・ろっくの作者の年齢は？")
print(response)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 まずは、それれの作品の出版年代をべてみよう。
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 では、どの作品から始めようか？
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

...

 それから、その作品がどの年代に出版されたかをべてみよう。
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 では、その作品のあるページをしてみよう。
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 それから、その作品がどの年代に出版されたかをべてみよう。
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

> Finished chain.
Agent stopped due to iteration limit or time limit.


In [12]:
%time
response = agent.run("最後のジェダイは誰？")
print(response)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 このにえるためには、画や小などの情報源を使用する必要があります。
Action: Search
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Llama.generate: prefix-match hit


 しありませんが、この回は正しい形式ではありません。必要な情報を提してください。
Final Answer: 最後のジェダイはルーク・スカイウォーカーです。

> Finished chain.
最後のジェダイはルーク・スカイウォーカーです。


In [13]:
%time
response = agent.run("Who is the last JEDI？")
print(response)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 There are no Jedis anymore. The Jedi Order was destroyed in the events of Star Wars: Episode III - Revenge of the Sith.
Action: Search
Action Input: "last jedi"
Observation: [snippet: A 2017 epic space opera film and the second installment of the Star Wars sequel trilogy. The film follows Rey as she seeks the aid of Luke Skywalker, who is under the influence of the dark side, while the Resistance faces a First Order attack. The film features the first posthumous film performance by Carrie Fisher and the first appearance of the Force Awakens characters., title: Star Wars: The Last Jedi - Wikipedia, link: https://en.wikipedia.org/wiki/Star_Wars:_The_Last_Jedi], [snippet: Star Wars: The Last Jedi (also known as Star Wars Episode VIII: The Last Jedi) is the eighth live-action theatrical installment in the Star Wars film franchise and the second installment in the sequel trilogy. Produced under the Disney-Lucasfilm brand, it's the tenth theatrically released film in the Star Wars franchise

Llama.generate: prefix-match hit


 I now know that the last Jedi is a character named Rey.
Final Answer: Rey is the last Jedi.

> Finished chain.
Rey is the last Jedi.


In [14]:
%time
response = agent.run("大谷翔平は、2023年のシーズンでは何本のホームランを打ったか？")
print(response)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 I need to find out how many home runs Tatsuya Ogawa hit in the 2023 season.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


 I need to search for information on Tatsuya Ogawa's home run count for the 2023 season.
Action: Search
Action Input: "Tatsuya Ogawa 2023 home runs"
Observation: [snippet: With his win, Shohei Kimura locked in a clean sweep of all three Pokémon VGC brackets for Japan at the 2023 Pokémon World Championships alongside Tomoya Ogawa in the Senior Division and Sora ..., title: Pokémon World Championships 2023: All live results, top finishers, and ..., link: https://dotesports.com/pokemon/news/pokemon-world-championships-2023-all-live-results-top-finishers-and-big-announcements], [snippet: As the 2023 MLB season approaches, fans and bettors alike are speculating the home run totals of several key players. DraftKings Sportsbook has posted the corresponding over/unders, and we'll take a closer look at several that jump off the screen for better or worse. 2023 MLB HR total props Shohei Ohtani, Under 35.5 HRs (-120), title: MLB predictions 2023: Best bets, picks for HR total player props this ..

...

Llama.generate: prefix-match hit


 I found some information on Tatsuya Ogawa's home run count for the 2023 season, but it is not accurate or reliable.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


 I need to search for more accurate and reliable information on Tatsuya Ogawa's home run count for the 2023 season in baseball.
Action: Search
Action Input: "Tatsuya Ogawa 2
Observation: [snippet: Tatsuya Ogawa. Animation Department: Catwoman: Hunted. Tatsuya Ogawa is known for Catwoman: Hunted (2022). Menu. Movies. Release Calendar Top 250 Movies Most Popular Movies Browse Movies by Genre Top Box Office Showtimes & Tickets Movie News India Movie Spotlight. TV Shows., title: Tatsuya Ogawa - IMDb, link: https://www.imdb.com/name/nm13392445/], [snippet: Work in Progress 27 Mar 2023 Graphic Depictions Of Violence Major Character Death Original Character x Original Character Original Female Character (s) Original Male Character (s) Akemi Suzuki (Ultimate Herbalist) Tao Manakorisen (Ultimate Barista) Zakoshi Natsuro (Ultimate Paranormal Investigator) Pon Palorstep (Ultimate Exorcist), title: Tatsuya Ogawa (Ultimate Lucky Student) | Archive ... - Archive of Our Own, link: https://archiveofou

ValueError: ignored

In [15]:
%%time
response = agent.run("大谷翔平は、2023年のシーズンでは何本のホームランを打っているか？また、最終的に何本打つと予測できるか？")
print(response)



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 これは野球選手の成をべるです。大谷平が2023年のシーズンで何本のホームランを打っているか、また最的に何本打つとできるかどうかを知りたいと思います。
Action: Search
Action Input: 大谷平の2023年シーズンのホームラン数
Observation: [snippet: エンゼルス大谷翔平の今シーズンの成績・活躍まとめ｜ホームランや勝利数は Sporting News Japan Staff • 読了時間 約16分 Getty Images 2023年シーズン開幕前に行われたワールド・ベースボール・クラシックでは、もはや代名詞となった"二刀流"の活躍で、侍ジャパンを世界一へと導いた大谷翔平。..., title: エンゼルス大谷翔平の今シーズンの成績・活躍まとめ｜ホームランや勝利数は - スポーティングニュース, link: https://www.sportingnews.com/jp/mlb/news/shohei-ohtani-2023-profile/sfuop36a8w2sbthsabnzjudw], [snippet: — MLB (@MLB) August 13, 2023 8月3日 マリナーズ戦 40号 【 #エンゼルス 】 #大谷翔平 登板試合で放った第40号ホームラン😤 今季は再びMVP獲得のシーズンか？ ？ 🤩 #日本人選手情報 https://t.co/qn6NFmc8wL pic.twitter.com/drNR58QY0I — MLB Japan (@MLBJapan)..., title: 大谷翔平 2023年の全ホームランを動画で振り返る - 日本経済新聞, link: https://www.nikkei.com/telling/DGXZTS00005160Q3A620C2000000/], [snippet: 2023年7月29日 14時58分 大谷 翔平. 大リーグ、エンジェルスの大谷翔平選手が28日、ブルージェイズ戦で、前日の試合から3打席連続となる39号ホーム ..., title: 大谷翔平 39号先制ホームラン 自身初の3打席連続 | Nhk, link: https://www3.nhk.or.jp/news/html/20230729/k10014146361000.html], [snip

Llama.generate: prefix-match hit


 これらの情報から、大谷平は2023年シーズンで51本のホームランを打っているとできます。
Final Answer: 大谷平は2023年シーズンで51本のホームランを打っています。

> Finished chain.
大谷平は2023年シーズンで51本のホームランを打っています。
CPU times: user 46.6 s, sys: 181 ms, total: 46.8 s
Wall time: 48.2 s


In [16]:
%%time
response = agent.run("2023年8月に上映されている日本映画のアニメのおススメを一つ選んでください？")
print(response)



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 まずは、最新の日本画情報をべてみましょう。
Action: Search
Action Input: "2023年8月に上される日本画アニメ"
Observation: [snippet: 映画『君たちはどう生きるか』 (2023年) ©2023 Studio Ghibli 公開日：2023年7月14日 (金) 『風立ちぬ』から10年ぶりとなる、宮崎駿の新作映画『君たちはどう生きるか』が公開。 宮崎駿が原作・脚本・監督を手がけ、スタジオジブリが製作を担う。 >>詳細はこちら 映画『ザ・スーパーマリオブラザーズ・ムービー』 (2023年) © 2023..., title: 2023年公開予定の＜アニメ映画＞話題作を一挙紹介!宮崎駿の最新作や人気アニメの劇場版 - ファッションプレス, link: https://www.fashion-press.net/news/87697], [snippet: 2023年に順次公開されるアニメ映画。 おすすめ作品の公開日やキャスト、スタッフ情報などを50音順でご紹介します。 目次 『アイカツ! 10th STORY ～未来への STARWAY～』 『アイドルマスター ミリオンライブ! 』 『【推しの子】Mother and Children』 『映画 きかんしゃトーマス めざせ! 夢のチャンピオンカップ』 『鬼太郎誕生 ゲゲゲの謎』 目次を開く 『アイカツ! 10th STORY ～未来への STARWAY～』 ＜公開日＞ 2023年1月20日公開 【STAFF】 企画・原作・制作：BN Pictures 原案：バンダイ 監督：木村隆一 脚本：加藤陽一 キャラクターデザイン：やぐちひろこ 配給：バンダイナムコピクチャーズ, title: 2023年アニメ映画おすすめ一覧｜人気・注目作の公開予定も | インタビュー・特集 | Abema Times | アベマタイムズ, link: https://times.abema.tv/articles/-/10065340], [snippet: 【2023夏アニメ】 2023年6月～8月の春に放送予定の今期アニメ情報（新作アニメ）を一覧表で公開。 放映日順に並んでいるので番組予約に最適! 期間：2023年6月～8月まで 最終更新日：2023年6月30日（金）13:41 ※情報は随時

Llama.generate: prefix-match hit


 宮崎がしている『君たちはどう生きるか』が公開されるので、その他の宮崎作品をす。
Action: Search
Action Input: "宮崎 アニメ"
Observation: [snippet: 宮崎 駿 （または 宮﨑 駿 、みやざき はやお、 1941年 〈 昭和 16年〉 1月5日 - ）は、 日本 の アニメーション監督 ・ アニメーター ・ 脚本家 ・ 漫画家 。 別名義として 秋津 三朗 （あきつ さぶろう）、 照樹 務 （てるき つとむ または てれこむ）がある。 株式会社スタジオジブリ 取締役 、 公益財団法人 徳間記念アニメーション文化財団 理事長 、 三鷹市立アニメーション美術館 （ 三鷹の森ジブリ美術館 ）名誉館主（初代館主） [3] 。 東京府 東京市 出身。 学習院大学 政経学部 卒業。 1963年 に 東映動画 にアニメーターとして入社。, title: 宮崎駿 - Wikipedia, link: https://ja.wikipedia.org/wiki/宮崎駿], [snippet: 『 となりのトトロ 』（英題： My Neighbor Totoro ）は、 1988年 4月16日 に公開された スタジオジブリ 制作の長編 アニメーション 映画。 宮崎駿 監督作品。 昭和30年代前半の日本 を舞台にした ファンタジー 。 田舎へ引っ越してきた草壁一家の サツキ ・ メイ 姉妹と、子どもの時にしか会えないと言われる不思議な生き物・ トトロ との交流を描く。 キャッチコピー は「 このへんな生きものは まだ日本にいるのです。 たぶん。 」 [注 3] 。 同時上映は『 火垂るの墓 』。 ストーリー 昭和 30年代前半の初夏 (5月 [8] [注 4]) 。, title: となりのトトロ - Wikipedia, link: https://ja.wikipedia.org/wiki/となりのトトロ], [snippet: アニメーション映画界の巨匠、宮崎駿監督の新作長編アニメ映画「君たちはどう生きるか」が来年7月に公開されることになりました。 宮崎監督の新作長編映画は10年ぶりとなります。 これは映画会社の東宝が13日、東京 千代田区で会見を開いて発表しました。..., title: 宮崎駿監督 10年ぶり長編映画「君たちは

Llama.generate: prefix-match hit


 宮崎がしている『君たちはどう生きるか』が公開されるので、その他の宮崎作品をす。
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


 I now know the final answer.
Final Answer: 宮崎が手がける2023年公開のアニメ画は、『君たちはどう生きるか』です。

> Finished chain.
宮崎が手がける2023年公開のアニメ画は、『君たちはどう生きるか』です。
CPU times: user 1min 50s, sys: 229 ms, total: 1min 51s
Wall time: 1min 53s
